# 1.2 Recomendadores basados en memoria

Se habla de recomendadores basados en memoria porque requieren mantener todos los datos disponibles para poder realizar una recomendación; además, cada vez que se introduce un nuevo dato o que se solicita una recomendación estos recomendadores han de efectuar sus operaciones sobre el conjunto completo de datos, lo que provoca que sean relativamente ineficientes, especialmente en aplicaciones con volúmenes de datos grandes. No obstante, tienen interés porque son muy sencillos conceptualmente y su programación es relativamente rápida.

## 1.2.1 Aproximaciones simples

La forma más sencilla de generar una valoración global de un conjunto de elemenos es promediar su grado de valoración. En el ejemplo de las películas, si una película tiene cuatro valoraciones (3, 5, 4, 3), su valoración media será 3,75; las películas con mayor valoración global serán las sugeridas al usuario.

In [1]:
import pandas as pd

In [2]:
ratings = pd.read_csv('data/ml-latest-small/ratings.csv')

ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
from collections import defaultdict

movie_rating = defaultdict(list)

In [4]:
%%time
for rate in ratings.iloc:
    movie_rating[rate['movieId']].append(rate['rating'])

CPU times: user 17.8 s, sys: 566 ms, total: 18.3 s
Wall time: 20.4 s


In [5]:
movie_global_rate = [ 
    (movieId, sum(movie_rating[movieId])/float(len(movie_rating[movieId])))
    for movieId in movie_rating 
]

In [6]:
movie_global_rate.sort(key = lambda x : x[1], reverse = True)

In [7]:
movie_global_rate[:10]

[(131724.0, 5.0),
 (5746.0, 5.0),
 (6835.0, 5.0),
 (3851.0, 5.0),
 (1151.0, 5.0),
 (1631.0, 5.0),
 (2075.0, 5.0),
 (176601.0, 5.0),
 (92494.0, 5.0),
 (102217.0, 5.0)]

In [8]:
movie_global_rate[-10:]

[(102735.0, 0.5),
 (102749.0, 0.5),
 (104017.0, 0.5),
 (111785.0, 0.5),
 (122627.0, 0.5),
 (136297.0, 0.5),
 (145724.0, 0.5),
 (61818.0, 0.5),
 (72424.0, 0.5),
 (145951.0, 0.5)]

In [9]:
len(movie_global_rate)

9724

La principal limitación de este método radica en que no tiene en cuenta las preferencias individuales de cada usuario. No obstante, esta estrategia puede ser útil con usuarios nuevos, de los que no se conoce sus gustos. Una ventaja es que no depende de ninguna métrica para efectuar sus recomendaciones.

## 1.2.2 Recomendación ponderada

La idea es sencilla: se trata de obtener una valoración global del conjunto de elementos (productos) pero en lugar de calcular la media aritmética de las valoraciones, hay que calcular la media de las valoraciones ponderada por la afinidad del usuario correspondiente. De esa manera, la valoración global de los productos está personalizada para cada usuario y, por tanto, debería resultar más útil y acertada.

$vp_j(x)$ es la valoración ponderada de una película $x$ para el usuario $j$, y viene dada por la expresión:

![valoracion_ponderada](images/valoracion_ponderada.png)

donde $v_i(x)$ es la valoración de la película $x$ por el usuario $i$ y $s(i,j)$ es la similitud entre los usuarios $i$ y $j$. Por último, $V(x)$ es el conjunto de usuarios que han valorado la película $x$.

In [10]:
%%time

users_ratings = defaultdict(dict)

for rate in ratings.iloc:
    users_ratings[rate['userId']][rate['movieId']] = rate['rating'] 

CPU times: user 16.9 s, sys: 489 ms, total: 17.4 s
Wall time: 18 s


In [11]:
from math import sqrt

In [12]:
def coefPearson(dic1: dict, dic2: dict) -> float:
    # Obtener los elementos comunes a los dos diccionarios
    common = [x for x in dic1 if x in dic2]
    n = float(len(common))
    
    # Si no hay elementos comunes, se devuelve cero; si no
    # se calcula el coeficiente
    if n == 0:
        return 0
    
    # Cálculo de las medias de cada diccionario
    mean1 = sum([dic1[x] for x in common])/n
    mean2 = sum([dic2[x] for x in common])/n
    
    # Cálculo del numerador y del denominador
    num = sum([(dic1[x] - mean1)*(dic2[x] - mean2) for x in common])
    
    den1 = sqrt(sum([pow((dic1[x] - mean1),2) for x in common]))
    den2 = sqrt(sum([pow((dic2[x] - mean2),2) for x in common]))
    
    den = den1*den2
    
    # Calculo del coeficiente si es posible, o devuelve 0
    if den == 0:
        return 0
    
    return num/den

In [13]:
def valoracionPonderada(usersRatings: dict, userId: int, simil = coefPearson) -> list:
    """
        Genera una lista ordenada de valoraciones ponderadas a partir 
        de un diccionario de valoraciones de usuarios y un id de usuario.
        Se puede elegir la función de similitud a utilizar
    """
    
    # En primer lugar se genera un diccionario con las similitudes 
    # del usuario [userId] con todos los demás
    # Este diccionario podría almacenarse para evitar recalcularlo
    simils = { 
        x: simil(usersRatings[userId], usersRatings[x]) 
        for x in usersRatings if x != userId 
    }
    
    # Diccionarios auxiliares
    # { movieId: [valoracion * similitud usuarios] }
    # { movieId: [similitud usuarios] }
    # numerador y denominador de la valoración ponderada
    
    num = defaultdict(list)
    den = defaultdict(list)
    
    # Se recorre el diccionario de valoraciones y se rellenan 
    # los diccionarios auxiliares con los valores encontrados
    for userId in simils:
        for movieId in usersRatings[userId]:
            
            num[movieId].append(
                usersRatings[userId][movieId]*simils[userId]
            )
            
            den[movieId].append(simils[userId])
            
    # Se calculan y ordenan las valoraciones ponderadas
    result = []
    for movieId in num:
        s1 = sum(num[movieId])
        s2 = sum(den[movieId])
        
        if s2 == 0:
            mean = 0
        else:
            mean = s1/s2
            
        result.append((movieId, mean))
        
    result.sort(key = lambda x: x[1], reverse = True)
    
    return result

In [14]:
%%time
rating_range = valoracionPonderada(users_ratings, 10)

CPU times: user 190 ms, sys: 11.5 ms, total: 201 ms
Wall time: 204 ms


In [15]:
rating_range[:10]

[(85788.0, 1670.8675517089694),
 (81417.0, 418.80799013817307),
 (80880.0, 352.6259699868691),
 (3594.0, 109.99149199617712),
 (60.0, 108.45349422246125),
 (2759.0, 96.57064452774712),
 (2500.0, 94.2367475077849),
 (339.0, 80.91652896148034),
 (2496.0, 76.21049606159426),
 (1272.0, 72.24006304234383)]

In [16]:
rating_range[-10:]

[(325.0, -42.795143737797574),
 (26171.0, -44.511793342212584),
 (54648.0, -45.98945526940052),
 (4471.0, -65.58011834579297),
 (79.0, -65.58548021127913),
 (519.0, -81.33370525681559),
 (3498.0, -95.65524500553492),
 (6643.0, -124.63207926747123),
 (1670.0, -164.64580396220637),
 (2574.0, -205.33413427217852)]

## Conclusiones

Los recomendadores vistos hasta el momento permiten mejorar de una forma muy sencilla sitios web y otras aplicaciones similares para sintonizar mejor con los usuarios y sugerirles productos que les puedan interesar, mejorando así tanto su satisfacción con la aplicación como los éxitos potenciales de venta o acceso.

Las limitaciones de los métodos vistos hasta ahora son que, como se ha explicado, son métodos basados en memoria: requieren almacenar y procesar todos los datos cada vez que se realiza una consulta. De este modo, una operación sencilla no resulta excesivamente costosa, pero como se tiene que repetir
completamente en cada consulta puede dar lugar a una carga computacional y de memoria desmesurada en aplicaciones medianas y grandes.

Otra limitación fundamental es que estos métodos de recomendación no abstraen los datos de ninguna forma, es decir, no proporcionan información global acerca de los datos de los que se dispone, con lo que su uso está limitado a producir recomendaciones de productos, pero no pueden utilizarse para abordar estudios más complejos sobre los tipos de usuarios o productos de que se dispone, estudios que son esenciales para analizar el funcionamiento de la aplicación y diseñar su futuro. 

**Los algoritmos de agrupamiento (clustering) sí abstraen y producen información de alto nivel a partir de los datos disponibles.**